In [60]:
#!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
#!unzip ml-100k.zip
!unzip ml-1m.zip

--2020-02-16 13:41:07--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

100%[======================================>] 5,917,549   11.8MB/s   in 0.5s   

2020-02-16 13:41:07 (11.8 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import pandas as pd
import torch as torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import time
#from torch.utils.tensorboard import SummaryWriter
#from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px
import math
from torch.utils.data import Dataset
import itertools

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
torch.cuda.device_count()

4

In [8]:
movies = pd.read_csv('ml-1m/movies.dat',sep='::',names=['movieId','title','genres'],encoding='latin-1')
movies['movieId_codes'] = movies['movieId'].astype('category').cat.codes
chain = itertools.chain(*[i.split('|') for i in movies['genres']])
genres = dict(enumerate(set(chain)))
list(genres.items())

/home/yonatang/nlp_yelpreview_techniques/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



[(0, 'Animation'),
 (1, 'Fantasy'),
 (2, 'Thriller'),
 (3, 'Western'),
 (4, 'Action'),
 (5, 'Documentary'),
 (6, 'Comedy'),
 (7, 'Horror'),
 (8, "Children's"),
 (9, 'Sci-Fi'),
 (10, 'Crime'),
 (11, 'Film-Noir'),
 (12, 'War'),
 (13, 'Mystery'),
 (14, 'Romance'),
 (15, 'Musical'),
 (16, 'Adventure'),
 (17, 'Drama')]

In [9]:
# 1 hot for every genre
for k,v in genres.items():
    movies[v] = movies['genres'].apply(lambda g: 1 if v in g else 0)

In [10]:
movies

,movieId,title,genres,movieId_codes,Animation,Fantasy,Thriller,Western,Action,Documentary,...,Children's,Sci-Fi,Crime,Film-Noir,War,Mystery,Romance,Musical,Adventure,Drama
0,1,Toy Story (1995),Animation|Children's|Comedy,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,3,Grumpier Old Men (1995),Comedy|Romance,2,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,Father of the Bride Part II (1995),Comedy,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,3878,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream (2000),Drama,3879,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3880,3950,Tigerland (2000),Drama,3880,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3881,3951,Two Family House (2000),Drama,3881,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
ratings=pd.read_csv('ml-1m/ratings.dat',sep='::',names=['userId','movieId','rating','time'])
ratings=ratings.join(movies.set_index('movieId'),on='movieId')

/home/yonatang/nlp_yelpreview_techniques/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [12]:
ratings

,userId,movieId,rating,time,title,genres,movieId_codes,Animation,Fantasy,Thriller,...,Children's,Sci-Fi,Crime,Film-Noir,War,Mystery,Romance,Musical,Adventure,Drama
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1176,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,655,1,0,0,...,1,0,0,0,0,0,0,1,0,0
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,902,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,3339,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,2286,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,1075,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,1078,0,0,0,...,0,0,0,0,1,0,1,0,0,1
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,558,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,1080,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None, names=['userId', 'gender' ,'age','occupation', 'zipcode'])
users['gender_codes'] = users['gender'].astype('category').cat.codes
users['age_codes'] = users['age'].astype('category').cat.codes
users['occupation_codes'] = users['occupation'].astype('category').cat.codes
users['userId_codes'] = users['userId'].astype('category').cat.codes

/home/yonatang/nlp_yelpreview_techniques/venv/lib64/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [14]:
users

,userId,gender,age,occupation,zipcode,gender_codes,age_codes,occupation_codes,userId_codes
0,1,F,1,10,48067,0,0,10,0
1,2,M,56,16,70072,1,6,16,1
2,3,M,25,15,55117,1,2,15,2
3,4,M,45,7,02460,1,4,7,3
4,5,M,25,20,55455,1,2,20,4
...,...,...,...,...,...,...,...,...,...
6035,6036,F,25,15,32603,0,2,15,6035
6036,6037,F,45,1,76006,0,4,1,6036
6037,6038,F,56,1,14706,0,6,1,6037
6038,6039,F,45,0,01060,0,4,0,6038


In [15]:
ratings=ratings.join(users.set_index('userId'), on='userId')

In [16]:
ratings

,userId,movieId,rating,time,title,genres,movieId_codes,Animation,Fantasy,Thriller,...,Adventure,Drama,gender,age,occupation,zipcode,gender_codes,age_codes,occupation_codes,userId_codes
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1176,0,0,0,...,0,1,F,1,10,48067,0,0,10,0
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,655,1,0,0,...,0,0,F,1,10,48067,0,0,10,0
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,902,0,0,0,...,0,0,F,1,10,48067,0,0,10,0
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,3339,0,0,0,...,0,1,F,1,10,48067,0,0,10,0
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,2286,1,0,0,...,0,0,F,1,10,48067,0,0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,1075,0,0,0,...,0,0,M,25,6,11106,1,2,6,6039
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,1078,0,0,0,...,0,1,M,25,6,11106,1,2,6,6039
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,558,0,0,0,...,0,1,M,25,6,11106,1,2,6,6039
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,1080,0,0,0,...,0,1,M,25,6,11106,1,2,6,6039


In [17]:
feature_columns = ['userId_codes','movieId_codes',*genres.values(),'age_codes','gender_codes','occupation_codes']
features_width = {
    'userId_codes':len(ratings['userId_codes'].unique()),
    'movieId_codes':len(ratings['movieId_codes'].unique()),
    **{i:1 for i in genres.values()},
    'age_codes':len(ratings['age_codes'].unique()),
    'gender_codes':len(ratings['gender_codes'].unique()),
    'occupation_codes':len(ratings['occupation_codes'].unique()),
}

In [18]:
features_width

{'userId_codes': 6040,
 'movieId_codes': 3706,
 'Animation': 1,
 'Fantasy': 1,
 'Thriller': 1,
 'Western': 1,
 'Action': 1,
 'Documentary': 1,
 'Comedy': 1,
 'Horror': 1,
 "Children's": 1,
 'Sci-Fi': 1,
 'Crime': 1,
 'Film-Noir': 1,
 'War': 1,
 'Mystery': 1,
 'Romance': 1,
 'Musical': 1,
 'Adventure': 1,
 'Drama': 1,
 'age_codes': 7,
 'gender_codes': 2,
 'occupation_codes': 21}

In [19]:
ratings[feature_columns]

,userId_codes,movieId_codes,Animation,Fantasy,Thriller,Western,Action,Documentary,Comedy,Horror,...,Film-Noir,War,Mystery,Romance,Musical,Adventure,Drama,age_codes,gender_codes,occupation_codes
0,0,1176,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,10
1,0,655,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,10
2,0,902,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,10
3,0,3339,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,10
4,0,2286,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6039,1075,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,2,1,6
1000205,6039,1078,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,2,1,6
1000206,6039,558,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,2,1,6
1000207,6039,1080,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2,1,6


In [20]:
# calculate offsets
#start from 1, 0 is padding index
next_offset = 1
features_offsets={}
for k,v in features_width.items():
    features_offsets[k] = next_offset
    next_offset += v

In [21]:
# map all column indides to start from correct offset
for categorical_column in ['userId_codes','movieId_codes','age_codes','gender_codes','occupation_codes']:
    ratings[categorical_column] = ratings[categorical_column].apply(lambda c: c + features_offsets[categorical_column])

for indicator_column in genres.values():
    ratings.loc[:,indicator_column] *= features_offsets[indicator_column]

In [22]:
ratings[feature_columns]

,userId_codes,movieId_codes,Animation,Fantasy,Thriller,Western,Action,Documentary,Comedy,Horror,...,Film-Noir,War,Mystery,Romance,Musical,Adventure,Drama,age_codes,gender_codes,occupation_codes
0,1,7217,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,9764,9765,9772,9784
1,1,6696,9747,0,0,0,0,0,0,0,...,0,0,0,0,9762,0,0,9765,9772,9784
2,1,6943,0,0,0,0,0,0,0,0,...,0,0,0,9761,9762,0,0,9765,9772,9784
3,1,9380,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,9764,9765,9772,9784
4,1,8327,9747,0,0,0,0,0,9753,0,...,0,0,0,0,0,0,0,9765,9772,9784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,7116,0,0,0,0,0,0,9753,0,...,0,0,0,0,0,0,0,9767,9773,9780
1000205,6040,7119,0,0,0,0,0,0,0,0,...,0,9759,0,9761,0,0,9764,9767,9773,9780
1000206,6040,6599,0,0,0,0,0,0,9753,0,...,0,0,0,0,0,0,9764,9767,9773,9780
1000207,6040,7121,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,9764,9767,9773,9780


In [18]:
# create mapping from 
#dict( zip( movies['movieId'].cat.codes, df['title'] ) )

In [19]:
# This will bring us back to matrix factorization:
#X = torch.tensor(ratings[['userId_codes','movieId_codes']].values)
#data_x = X

In [23]:
# TODO yonigo - minimize matrix width to biggest row
X = torch.tensor(ratings[feature_columns].values)
data_x = torch.zeros_like(X)
for i in range(X.shape[0]):
    r = X[i,:]
    non_zeros = r[r!=0]
    data_x[i,0:(non_zeros.shape[0])] = non_zeros

In [24]:
data_y = torch.tensor(ratings['rating'].values).float()
dataset = data.TensorDataset(data_x, data_y)

In [25]:
bs=1024
train_n = int(len(dataset)*0.9)
valid_n = len(dataset) - train_n
splits = [train_n,valid_n]
assert sum(splits) == len(dataset)
trainset,devset = torch.utils.data.random_split(dataset,splits)
train_dataloader = data.DataLoader(trainset,batch_size=bs,shuffle=True,num_workers=0)
dev_dataloader = data.DataLoader(devset,batch_size=bs,shuffle=True,num_workers=0)

In [26]:
# copied from fastai: 
def trunc_normal_(x, mean=0., std=1.):
    "Truncated normal initialization."
    # From https://discuss.pytorch.org/t/implementing-truncated-normal-initializer/4778/12
    return x.normal_().fmod_(2).mul_(std).add_(mean)

In [113]:
class NNet(nn.Module):
    def __init__(self, n, k):
        super().__init__()

        self.w0 = nn.Parameter(torch.zeros(1))
        self.bias = nn.Embedding(n, 1)
        self.embeddings = nn.Embedding(n, k, padding_idx=0) # this is V matrix from paper

        # See https://arxiv.org/abs/1711.09160
        with torch.no_grad(): trunc_normal_(self.embeddings.weight, std=0.01)
        with torch.no_grad(): trunc_normal_(self.bias.weight, std=0.01)

    def forward(self, X):
        emb = self.embeddings(X)

        # zero the embeddings of padding indices
        mask = X != 0
        emb = emb*(mask.unsqueeze(2))

        # calculate the interactions in complexity of O(nk) see lemma 3.1 from paper
        a = emb.sum(dim=1).pow(2)
        b = emb.pow(2).sum(dim=1)
        pairwise = (a-b).sum(1)*0.5
        bias = self.bias(X).squeeze().sum(1)
        return torch.sigmoid(self.w0 + bias + pairwise)*5.5


In [114]:
# fit/test functions
def fit(iterator, model, optimizer, criterion):
    train_loss = 0
    model.train()
    for x,y in iterator:
        optimizer.zero_grad()
        y_hat = model(x.to(device))
        loss = criterion(y_hat, y.to(device))
        train_loss += loss.item()*x.shape[0]
        loss.backward()
        optimizer.step()
    return train_loss / len(iterator.dataset)

def test(iterator, model, criterion):
    train_loss = 0
    model.eval()
    for x,y in iterator:                    
        with torch.no_grad():
            y_hat = model(x.to(device))
        loss = criterion(y_hat, y.to(device))
        train_loss += loss.item()*x.shape[0]
    return train_loss / len(iterator.dataset)

In [115]:
def train_n_epochs(model, n, optimizer):

    criterion = nn.MSELoss().to(device)
    for epoch in range(n):
        start_time = time.time()
        train_loss = fit(train_dataloader, model, optimizer, criterion)
        valid_loss = test(dev_dataloader, model, criterion)
        secs = int(time.time() - start_time)
        print(f'Epoch: {epoch} time: {secs}[s]')
        print(f'\tTrain Loss: {(train_loss):.4f}')
        print(f'\tValidation Loss: {(valid_loss):.4f}')

In [116]:
features = data_x.max() + 1
# TODO YONIGO - nn.DataParallel to use all GPU's
model = NNet(features,50).to(device)
wd=1e-5
lr=0.001
epochs=10
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
train_n_epochs(model,epochs,optimizer)

Epoch: 0 time: 22[s]
	Train Loss: 0.8938
	Validation Loss: 0.8256
Epoch: 1 time: 22[s]
	Train Loss: 0.8029
	Validation Loss: 0.7973
Epoch: 2 time: 22[s]
	Train Loss: 0.7643
	Validation Loss: 0.7721
Epoch: 3 time: 22[s]
	Train Loss: 0.7299
	Validation Loss: 0.7553
Epoch: 4 time: 22[s]
	Train Loss: 0.6982
	Validation Loss: 0.7435
Epoch: 5 time: 22[s]
	Train Loss: 0.6689
	Validation Loss: 0.7336
Epoch: 6 time: 22[s]
	Train Loss: 0.6423
	Validation Loss: 0.7310
Epoch: 7 time: 22[s]
	Train Loss: 0.6184
	Validation Loss: 0.7273
Epoch: 8 time: 22[s]
	Train Loss: 0.5957
	Validation Loss: 0.7285
Epoch: 9 time: 22[s]
	Train Loss: 0.5751
	Validation Loss: 0.7298


## Recommend best genre for a new woman

In [117]:
# now lets see what genres are best to recommend to a woman
woman_code = 0
woman_offset = woman_code + features_offsets['gender_codes']
woman_embeddings = model.embeddings(torch.tensor(woman_offset,device=device))
#woman_bias = model.bias(torch.tensor(woman_offset))

In [119]:
woman_embeddings

tensor([ 0.0029, -0.0818, -0.0555,  0.0482, -0.0501, -0.0116, -0.0490,  0.0221,
        -0.1119, -0.0092,  0.0009,  0.1004, -0.0561,  0.0623,  0.0109,  0.0996,
        -0.1959,  0.0141, -0.0060, -0.1000,  0.0366,  0.0389, -0.0424,  0.0674,
         0.0926,  0.0162, -0.0666, -0.0640, -0.0230,  0.1878,  0.2222,  0.0220,
        -0.0605,  0.1259,  0.0443, -0.0479, -0.0215, -0.0086, -0.0368,  0.1825,
        -0.0689,  0.0200, -0.0559,  0.0134, -0.0224, -0.0999, -0.0241, -0.1145,
         0.0017,  0.0519], device='cuda:0', grad_fn=<EmbeddingBackward>)

In [120]:
# embeddings of genres:
genre_embeddings = model.embeddings(torch.tensor([features_offsets[i] for i in genres.values()],device=device))
genre_bias = model.bias(torch.tensor([features_offsets[i] for i in genres.values()],device=device))

In [121]:
dot_results = (genre_bias.squeeze()+(woman_embeddings*genre_embeddings).sum(1))
dot_results

tensor([-0.1362,  0.0238,  0.0476,  0.0896,  0.0064, -0.1577,  0.0401, -0.0426,
        -0.0556,  0.0931, -0.1017,  0.0328,  0.0572,  0.2102, -0.2625, -0.1186,
        -0.0066,  0.0487], device='cuda:0', grad_fn=<AddBackward0>)

In [124]:
#best movies recommended to woman:
[genres[i.item()] for i in dot_results.argsort(descending=True)]

['Drama',
 'Documentary',
 'Romance',
 'Musical',
 'War',
 'Mystery',
 'Thriller',
 'Comedy',
 'Film-Noir',
 'Adventure',
 'Crime',
 'Western',
 "Children's",
 'Action',
 'Fantasy',
 'Sci-Fi',
 'Animation',
 'Horror']

## Recommend best genre for a new man

In [125]:
man_code = 1
man_offset = man_code + features_offsets['gender_codes']
man_embeddings = model.embeddings(torch.tensor(man_offset,device=device))

In [126]:
dot_results = (genre_bias.squeeze()+(man_embeddings*genre_embeddings).sum(1))
dot_results

tensor([-0.0932,  0.0945,  0.0696,  0.0119, -0.0127, -0.0728,  0.0534,  0.0149,
        -0.1975,  0.0792, -0.0539,  0.0172, -0.0190,  0.1955, -0.1871, -0.1264,
         0.0238,  0.1179], device='cuda:0', grad_fn=<AddBackward0>)

In [127]:
[genres[i.item()] for i in dot_results.argsort(descending=True)]

['Drama',
 'War',
 'Film-Noir',
 'Documentary',
 'Mystery',
 'Thriller',
 'Crime',
 'Comedy',
 'Western',
 'Romance',
 'Adventure',
 'Musical',
 'Action',
 'Animation',
 'Sci-Fi',
 'Fantasy',
 'Horror',
 "Children's"]

## Recommend best genre for a new under_18

In [148]:
under_18_code = 0
under_18_offset = under_18_code + features_offsets['age_codes']
under_18_embeddings = model.embeddings(torch.tensor(under_18_offset,device=device))

In [149]:
dot_results = (genre_bias.squeeze()+(under_18_embeddings*genre_embeddings).sum(1))
dot_results

tensor([ 0.1705,  0.0045, -0.0256, -0.0114, -0.0604, -0.0199,  0.0299, -0.0016,
        -0.1706, -0.0337,  0.1843,  0.0316, -0.0199,  0.1165, -0.0818, -0.0214,
         0.0814,  0.1100], device='cuda:0', grad_fn=<AddBackward0>)

In [150]:
[genres[i.item()] for i in dot_results.argsort(descending=True)]

['Action',
 'Sci-Fi',
 'Drama',
 'War',
 'Crime',
 'Comedy',
 'Thriller',
 'Film-Noir',
 'Western',
 'Romance',
 'Musical',
 'Animation',
 'Fantasy',
 'Mystery',
 'Documentary',
 'Adventure',
 'Horror',
 "Children's"]

In [151]:
def get_genre_recommendation(user_meta):
    print(user_meta)

In [153]:
#  Age codes:
# "Under 18": 0
# "18-24":1
# "25-34":2
# "35-44":3
# "45-49":4
# "50-55":5
# "56+":6

# Gender Codes:
# Woman: 0
# Man:1

#Ocupation Codes:
dict( zip( users['occupation_codes'], users['occupation'] ) )

{10: 10,
 16: 16,
 15: 15,
 7: 7,
 20: 20,
 9: 9,
 1: 1,
 12: 12,
 17: 17,
 0: 0,
 3: 3,
 14: 14,
 4: 4,
 11: 11,
 8: 8,
 19: 19,
 2: 2,
 18: 18,
 5: 5,
 13: 13,
 6: 6}